<a   href="https://colab.research.google.com/github/eduardojdiniz/Buzznauts/blob/master/scripts/load_videos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Example loader

### Import Buzznauts GitHub Repository and Install Dependencies

In [1]:
!pip install duecredit --quiet
!pip install nilearn --quiet
!pip install numpy --quiet
!pip install matplotlib --quiet
!pip install nibabel --quiet
!pip install decord --quiet
!pip install tqdm --quiet


!pip install opencv --quiet
!git clone https://github.com/eduardojdiniz/Buzznauts --quiet
!git clone https://github.com/Neural-Dynamics-of-Visual-Cognition-FUB/Algonauts2021_devkit.git

     |████████████████████████████████| 86 kB 3.1 MB/s 
     |████████████████████████████████| 179 kB 8.6 MB/s 
     |████████████████████████████████| 4.9 MB 5.0 MB/s 
     |████████████████████████████████| 13.6 MB 75 kB/s 
ERROR: Could not find a version that satisfies the requirement opencv (from versions: none)
ERROR: No matching distribution found for opencv
Cloning into 'Algonauts2021_devkit'...
remote: Enumerating objects: 68, done.
remote: Counting objects: 100% (68/68), done.
remote: Compressing objects: 100% (57/57), done.
remote: Total 68 (delta 33), reused 38 (delta 10), pack-reused 0
Unpacking objects: 100% (68/68), done.


### Import Buzznauts Module 

In [11]:
import Buzznauts.Buzznauts as buzz
# Imports
import os
import glob
import pickle
import torch

import numpy as np
import matplotlib.pyplot as plt
import nibabel as nib

from nilearn import datasets
from nilearn import plotting
from tqdm import tqdm

fsaverage = datasets.fetch_surf_fsaverage()

from google.colab import drive
drive.mount("/content/drive")

import os.path as op
videos = "/content/drive/MyDrive/Buzznauts/data/stimuli/videos"
dataframe = op.join(stimuli, "frames")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [12]:
from decord import VideoReader
from decord import cpu, gpu
from tqdm import tqdm
from random import randrange

def load_videos(path_to_videos, sample_strategy='standard', frame_n=3, win_n=3):
    vid_list = os.listdir(path_to_videos)
    vid_list.sort()
    mp4_filename_format = "{path}/{filename}"
    mp4_path_files = glob.glob(path_to_videos + '*.mp4')
    X = {}
    i = 0

    with tqdm(total = len(vid_list), position = 0, leave = True) as pbar:
        for video in tqdm(vid_list, position = 0, leave = True):
            mp4_filename = mp4_filename_format.format(path=path_to_videos, 
                                                filename=video)
            vr = VideoReader(mp4_filename, ctx=cpu(0))
      
            if sample_strategy=='standard':
                num_frames = len(vr)
                X[i] = vr.get_batch([0, int(num_frames*0.5), num_frames-1]).asnumpy()
            elif sample_strategy=='random':
                sample_n = np.zeros((frame_n))
                for x in range(sample_n.shape[0]):
                    sample_n[x] = randrange(0, len(vr))
                sample_n = sample_n.astype(int)
                X[i] = vr.get_batch([sample_n]).asnumpy()
            elif sample_strategy=='average':
                chunks = []
                start=0
                X[i]={}
                for chunk in range(win_n):
                    chunks.append(list(range(start, int(start + int(len(vr)/(win_n))))))
                    tmp_chunk = np.mean(vr.get_batch([chunks[chunk]]).asnumpy(), axis=0)
                    start =+  chunks[chunk][-1] + 1
                    X[i][chunk] = np.mean(vr.get_batch([chunks[chunk]]).asnumpy(), axis=0).astype(int)

            i += 1
    return X


#X = load_videos('/content/AlgonautsVideos268_All_30fpsmax/', sample_strategy='random', frame_n=4)
X = load_videos(videos, sample_strategy='average', win_n=2)

  0%|          | 0/1102 [00:33<?, ?it/s]


KeyboardInterrupt: ignored

In [ ]:
video2plot = 1
frame2plot = 2
imshow(X[video2plot][frame2plot])